In [7]:
import pandas as pd
import numpy as np

from utils import get_series_first_mode_or_nan

# Transactions

In [8]:
from utils import read_transactions

# transactions = read_transactions('data/samples/TRANSACTIONS_SAMPLE.csv')
transactions = read_transactions('data/initial/TRANSACTIONS.csv')
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15654626 entries, 0 to 15654625
Data columns (total 21 columns):
 #   Column     Dtype         
---  ------     -----         
 0   client_id  uint64        
 1   tran_date  datetime64[ns]
 2   cat_c2     int32         
 3   cat_c3     int32         
 4   cat_c4     int32         
 5   fl_c6      int8          
 6   fl_c7      int8          
 7   fl_c8      int8          
 8   fl_c9      int8          
 9   fl_c10     int8          
 10  fl_c11     int8          
 11  fl_c12     int8          
 12  fl_c13     int8          
 13  fl_c14     int8          
 14  fl_c15     int8          
 15  float_c16  float32       
 16  float_c17  float32       
 17  float_c18  float32       
 18  int_c19    int32         
 19  float_c20  float32       
 20  float_c21  float32       
dtypes: datetime64[ns](1), float32(5), int32(4), int8(10), uint64(1)
memory usage: 925.6 MB


In [9]:
# Date variables

tx_agg_dates = transactions.groupby('client_id').agg(
    tx_count=('tran_date', 'count'),
    tx_days=('tran_date', 'nunique'),
    first_tx_date=('tran_date', 'min'),
    last_tx_date=('tran_date', 'max'),
).reset_index()

tx_agg_dates['tx_avg_per_day'] = tx_agg_dates['tx_count'] / tx_agg_dates['tx_days']
tx_agg_dates['tx_period_days'] = (tx_agg_dates['last_tx_date'] - tx_agg_dates['first_tx_date']).dt.days

reference_date = transactions['tran_date'].max()
tx_agg_dates['days_since_last_tx'] = (reference_date - tx_agg_dates['last_tx_date']).dt.days

# Replace dates with days from starting point
min_date = transactions['tran_date'].min()
tx_agg_dates['first_tx_day'] = (tx_agg_dates['first_tx_date'] - min_date).dt.days
tx_agg_dates['last_tx_day'] = (tx_agg_dates['last_tx_date'] - min_date).dt.days
tx_agg_dates = tx_agg_dates.drop(columns=['first_tx_date', 'last_tx_date'])


In [10]:
# Numerical variables

num_cols = ['float_c16', 'float_c17', 'float_c18', 'int_c19', 'float_c20', 'float_c21']

tx_agg_nums = transactions.groupby('client_id')[num_cols].agg(['mean', 'std', 'min', 'max', 'sum'])
tx_agg_nums.columns = ['tx_' + '_'.join(col) for col in tx_agg_nums.columns]
tx_agg_nums = tx_agg_nums.reset_index()

# Handle NULL-values in "_std" columns
std_cols = [c for c in tx_agg_nums.columns if c.endswith('_std')]
tx_agg_nums['has_single_tx'] = tx_agg_nums[std_cols].isna().any(axis=1).astype('int8')
tx_agg_nums[std_cols] = tx_agg_nums[std_cols].fillna(0.0)


In [11]:
# Flag variables

flag_cols = [col for col in transactions.columns if col.startswith('fl_')]

tx_agg_flags = transactions.groupby('client_id')[flag_cols].mean()
tx_agg_flags.columns = [f'tx_{c}_ratio' for c in flag_cols]
tx_agg_flags = tx_agg_flags.reset_index()


In [12]:
# Categorical variables

cat_cols = ['cat_c2', 'cat_c3', 'cat_c4']
aggregates = {f'tx_{col}_nunique': (col, 'nunique') for col in cat_cols}
aggregates |= {f'tx_{col}_mode': (col, get_series_first_mode_or_nan) for col in cat_cols}

tx_agg_cat = transactions.groupby('client_id').agg(**aggregates).reset_index()


In [13]:
# Merge all features

tx_features = (
    tx_agg_dates
    .merge(tx_agg_nums, on='client_id', how='left')
    .merge(tx_agg_flags, on='client_id', how='left')
    .merge(tx_agg_cat, on='client_id', how='left')
)
tx_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35070 entries, 0 to 35069
Data columns (total 55 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   client_id           35070 non-null  uint64 
 1   tx_count            35070 non-null  int64  
 2   tx_days             35070 non-null  int64  
 3   tx_avg_per_day      35070 non-null  float64
 4   tx_period_days      35070 non-null  int64  
 5   days_since_last_tx  35070 non-null  int64  
 6   first_tx_day        35070 non-null  int64  
 7   last_tx_day         35070 non-null  int64  
 8   tx_float_c16_mean   35070 non-null  float32
 9   tx_float_c16_std    35070 non-null  float32
 10  tx_float_c16_min    35070 non-null  float32
 11  tx_float_c16_max    35070 non-null  float32
 12  tx_float_c16_sum    35070 non-null  float32
 13  tx_float_c17_mean   35070 non-null  float32
 14  tx_float_c17_std    35070 non-null  float32
 15  tx_float_c17_min    35070 non-null  float32
 16  tx_f

# App activity

In [16]:
from utils import handle_activity_null_values, read_app_activity

# activities = read_app_activity('data/samples/APP_ACTIVITY_SAMPLE.csv')
activities = read_app_activity('data/initial/APP_ACTIVITY.csv')
activities = handle_activity_null_values(activities)
activities.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 17665187 entries, 0 to 17738591
Data columns (total 13 columns):
 #   Column         Non-Null Count     Dtype         
---  ------         --------------     -----         
 0   client_id      17665187 non-null  uint64        
 1   device_id      17665187 non-null  uint64        
 2   activity_date  17665187 non-null  datetime64[ns]
 3   cat_c3         17665187 non-null  int32         
 4   cat_c4         17665187 non-null  int32         
 5   cat_c5         17665187 non-null  int32         
 6   cat_c6         17665187 non-null  int32         
 7   cat_c8         17665187 non-null  int8          
 8   cat_c9         17665187 non-null  int32         
 9   cat_c10        17665187 non-null  int8          
 10  float_c11      17665187 non-null  float32       
 11  float_c12      17665187 non-null  float32       
 12  float_c14      17665187 non-null  float32       
dtypes: datetime64[ns](1), float32(3), int32(5), int8(2), uint64(2)
memory usage

In [17]:
# Date variables

activities_agg_dates = activities.groupby('client_id').agg(
    activity_count=('activity_date', 'count'),
    activity_days=('activity_date', 'nunique'),
    first_activity_date=('activity_date', 'min'),
    last_activity_date=('activity_date', 'max'),
).reset_index()

activities_agg_dates['activity_avg_per_day'] = activities_agg_dates['activity_count'] / activities_agg_dates['activity_days']
activities_agg_dates['activity_period_days'] = (activities_agg_dates['last_activity_date'] - activities_agg_dates['first_activity_date']).dt.days

max_date = activities['activity_date'].max()
activities_agg_dates['days_since_last_activity'] = (max_date - activities_agg_dates['last_activity_date']).dt.days

# Replace dates with days from starting point
min_date = activities['activity_date'].min()
activities_agg_dates['first_activity_day'] = (activities_agg_dates['first_activity_date'] - min_date).dt.days
activities_agg_dates['last_activity_day'] = (activities_agg_dates['last_activity_date'] - min_date).dt.days
activities_agg_dates = activities_agg_dates.drop(columns=['first_activity_date', 'last_activity_date'])

In [18]:
# Numerical variables

num_cols = ['float_c11', 'float_c12', 'float_c14']

activity_agg_nums = activities.groupby('client_id')[num_cols].agg(['mean', 'std', 'min', 'max', 'sum'])
activity_agg_nums.columns = ['activity_' + '_'.join(col) for col in activity_agg_nums.columns]
activity_agg_nums = activity_agg_nums.reset_index()

# Handle NULL-values in "_std" columns
std_cols = [c for c in activity_agg_nums.columns if c.endswith('_std')]
activity_agg_nums['has_single_activity'] = activity_agg_nums[std_cols].isna().any(axis=1).astype('int8')
activity_agg_nums[std_cols] = activity_agg_nums[std_cols].fillna(0.0)


In [19]:
# Flag variables

flag_cols = ['cat_c8', 'cat_c10']

activity_agg_flags = activities.groupby('client_id')[flag_cols].mean()
activity_agg_flags.columns = [f'activity_{c}_ratio' for c in flag_cols]
activity_agg_flags = activity_agg_flags.reset_index()


In [20]:
# Categorical variables

cat_cols = ['cat_c3', 'cat_c4', 'cat_c5', 'cat_c6', 'cat_c9']
aggregates = {f'activity_{col}_nunique': (col, 'nunique') for col in cat_cols}
aggregates |= {f'activity_{col}_mode': (col, get_series_first_mode_or_nan) for col in cat_cols}

activity_agg_cat = activities.groupby('client_id').agg(**aggregates).reset_index()


In [21]:
# Extra features

activity_agg_extra = activities.groupby('client_id').agg(n_devices=('device_id', 'nunique')).reset_index()

In [22]:
# Merge all features

activity_features = (
    activities_agg_dates
    .merge(activity_agg_nums, on='client_id', how='left')
    .merge(activity_agg_flags, on='client_id', how='left')
    .merge(activity_agg_cat, on='client_id', how='left')
    .merge(activity_agg_extra, on='client_id', how='left')
)
activity_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35669 entries, 0 to 35668
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   client_id                 35669 non-null  uint64 
 1   activity_count            35669 non-null  int64  
 2   activity_days             35669 non-null  int64  
 3   activity_avg_per_day      35669 non-null  float64
 4   activity_period_days      35669 non-null  int64  
 5   days_since_last_activity  35669 non-null  int64  
 6   first_activity_day        35669 non-null  int64  
 7   last_activity_day         35669 non-null  int64  
 8   activity_float_c11_mean   35669 non-null  float32
 9   activity_float_c11_std    35669 non-null  float32
 10  activity_float_c11_min    35669 non-null  float32
 11  activity_float_c11_max    35669 non-null  float32
 12  activity_float_c11_sum    35669 non-null  float32
 13  activity_float_c12_mean   35669 non-null  float32
 14  activi

# Communications

In [1]:
from utils import encode_comm_categories, read_communications

# comms = read_communications('data/samples/COMMUNICATIONS_SAMPLE.csv')
comms = read_communications('data/initial/COMMUNICATIONS.csv')
comms = encode_comm_categories(comms)
comms.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 19586917 entries, 0 to 19586921
Data columns (total 6 columns):
 #   Column        Non-Null Count     Dtype         
---  ------        --------------     -----         
 0   client_id     19586917 non-null  uint64        
 1   contact_date  19586917 non-null  datetime64[ns]
 2   cat_c2        19586917 non-null  int32         
 3   cat_c3        19586917 non-null  int32         
 4   cat_c4        19586917 non-null  int32         
 5   cat_c5        19586917 non-null  int32         
dtypes: datetime64[ns](1), int32(4), uint64(1)
memory usage: 747.2 MB


In [ ]:
comms.head(10)

In [ ]:
comms.describe()

In [3]:
# Date variables

comms_agg_dates = comms.groupby('client_id').agg(
    comm_count=('contact_date', 'count'),
    comm_days=('contact_date', 'nunique'),
    first_comm_date=('contact_date', 'min'),
    last_comm_date=('contact_date', 'max'),
).reset_index()

comms_agg_dates['comm_avg_per_day'] = comms_agg_dates['comm_count'] / comms_agg_dates['comm_days']
comms_agg_dates['comm_period_days'] = (comms_agg_dates['last_comm_date'] - comms_agg_dates['first_comm_date']).dt.days

max_date = comms['contact_date'].max()
comms_agg_dates['days_since_last_comm'] = (max_date - comms_agg_dates['last_comm_date']).dt.days

# Replace dates with days from starting point
min_date = comms['contact_date'].min()
comms_agg_dates['first_comm_day'] = (comms_agg_dates['first_comm_date'] - min_date).dt.days
comms_agg_dates['last_comm_day'] = (comms_agg_dates['last_comm_date'] - min_date).dt.days
comms_agg_dates = comms_agg_dates.drop(columns=['first_comm_date', 'last_comm_date'])

In [4]:
# Categorical variables

aggregates = []
cat_cols = ['cat_c2', 'cat_c3', 'cat_c4', 'cat_c5']
aggregates = {f'comm_{col}_nunique': (col, 'nunique') for col in cat_cols}
aggregates |= {f'comm_{col}_mode': (col, get_series_first_mode_or_nan) for col in cat_cols}

comms_agg_cat = comms.groupby('client_id').agg(**aggregates).reset_index()

In [5]:
# Merge features
comms_features = comms_agg_dates.merge(comms_agg_cat, on='client_id', how='left')
comms_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35805 entries, 0 to 35804
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             35805 non-null  uint64 
 1   comm_count            35805 non-null  int64  
 2   comm_days             35805 non-null  int64  
 3   comm_avg_per_day      35805 non-null  float64
 4   comm_period_days      35805 non-null  int64  
 5   days_since_last_comm  35805 non-null  int64  
 6   first_comm_day        35805 non-null  int64  
 7   last_comm_day         35805 non-null  int64  
 8   comm_cat_c2_nunique   35805 non-null  int64  
 9   comm_cat_c3_nunique   35805 non-null  int64  
 10  comm_cat_c4_nunique   35805 non-null  int64  
 11  comm_cat_c5_nunique   35805 non-null  int64  
 12  comm_cat_c2_mode      35805 non-null  int32  
 13  comm_cat_c3_mode      35805 non-null  int32  
 14  comm_cat_c4_mode      35805 non-null  int32  
 15  comm_cat_c5_mode   

In [6]:
comms_features.describe()

,client_id,comm_count,comm_days,comm_avg_per_day,comm_period_days,days_since_last_comm,first_comm_day,last_comm_day,comm_cat_c2_nunique,comm_cat_c3_nunique,comm_cat_c4_nunique,comm_cat_c5_nunique,comm_cat_c2_mode,comm_cat_c3_mode,comm_cat_c4_mode,comm_cat_c5_mode
count,3.580500e+04,35805.000000,35805.000000,35805.000000,35805.000000,35805.000000,35805.000000,35805.000000,35805.000000,35805.000000,35805.000000,35805.000000,35805.000000,35805.000000,35805.000000,35805.000000
mean,9.231032e+18,547.044184,116.401341,4.038846,172.057953,50.060802,50.881246,222.939198,61.770144,3.376232,2.986510,5.101438,797.131825,3.996285,1.075353,3.992683
std,5.314770e+18,542.800941,47.713223,2.954254,28.943189,33.468066,43.625843,33.468066,18.146404,0.633055,0.131219,1.625280,112.026442,0.138066,0.270650,0.670785
min,8.446579e+14,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,9.000000,1.000000,1.000000,1.000000,1.000000,28.000000,1.000000,1.000000,0.000000
25%,4.660918e+18,176.000000,78.000000,2.183908,178.000000,31.000000,31.000000,181.000000,50.000000,3.000000,3.000000,4.000000,814.000000,4.000000,1.000000,4.000000
50%,9.214426e+18,387.000000,124.000000,3.209524,180.000000,62.000000,62.000000,211.000000,61.000000,3.000000,3.000000,5.000000,814.000000,4.000000,1.000000,4.000000
75%,1.380601e+19,741.000000,159.000000,4.967213,181.000000,92.000000,75.000000,242.000000,73.000000,4.000000,3.000000,6.000000,822.000000,4.000000,1.000000,4.000000
max,1.844580e+19,8732.000000,184.000000,104.071429,183.000000,264.000000,272.000000,273.000000,148.000000,7.000000,3.000000,14.000000,962.000000,8.000000,3.000000,76.000000


# Combine features from all datasets

In [23]:
client_features = (
    tx_features
    .merge(activity_features, on='client_id', how='inner')
    .merge(comms_features, on='client_id', how='inner')
)
# client_features.info(verbose=True, show_counts=True)

In [24]:
# Reduce float precision to float32
client_features = client_features.astype({col: 'float32' for col in client_features.select_dtypes(include=['float64']).columns})

# Downcast int64 to int32
for col in client_features.select_dtypes(include=['int64']).columns:
    min_val = client_features[col].min()
    max_val = client_features[col].max()
    if (min_val >= np.iinfo(np.int32).min) and (max_val <= np.iinfo(np.int32).max):
        client_features = client_features.astype({col: 'int32'})
    else:
        print(f'Column {col} exceeds int32 range')

In [25]:
client_features.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35048 entries, 0 to 35047
Data columns (total 106 columns):
 #    Column                    Non-Null Count  Dtype  
---   ------                    --------------  -----  
 0    client_id                 35048 non-null  uint64 
 1    tx_count                  35048 non-null  int32  
 2    tx_days                   35048 non-null  int32  
 3    tx_avg_per_day            35048 non-null  float32
 4    tx_period_days            35048 non-null  int32  
 5    days_since_last_tx        35048 non-null  int32  
 6    first_tx_day              35048 non-null  int32  
 7    last_tx_day               35048 non-null  int32  
 8    tx_float_c16_mean         35048 non-null  float32
 9    tx_float_c16_std          35048 non-null  float32
 10   tx_float_c16_min          35048 non-null  float32
 11   tx_float_c16_max          35048 non-null  float32
 12   tx_float_c16_sum          35048 non-null  float32
 13   tx_float_c17_mean         35048 non-null  fl

In [26]:
client_features.to_csv('data/features/features_2.csv', index=False)